### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import re
import tldextract
from ydata_profiling import ProfileReport
from fuzzywuzzy import fuzz
from collections import Counter

c:\Users\mohit\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


***
## ***Step 1 -Fetching Technology and Netloc From Links Fetched***
***

### Importing the links data

In [2]:
df = pd.read_csv('Step 0 - Raw Data/Website_Technography_Relationship.csv')

In [3]:
df.head()

,File Name,Link,Status
0,02june-fsa.com,https://static.wixstatic.com/media/,Present
1,02june-fsa.com,https://static.wixstatic.com/media/0525b5_c4e2...,Present
2,02june-fsa.com,./#comp-jbgap0ye,Present
3,02june-fsa.com,https://static.parastorage.com/unpkg/focus-wit...,Present
4,02june-fsa.com,https://panorama.wixapps.net/api/v1/bulklog,Present


***
***

# Raw Data to Netloc Unclean Table

### Cleaning the File Name and Link Columns

In [4]:
df_new = df.copy()

In [5]:
df_new['File Name'] = df_new['File Name'].str.lower()

In [6]:
df_new['Link'] = df_new['Link'].str.lower()

### Extrating Netloc Column

In [7]:
def extract_netloc(url):
    try:
        parsed_url = urlparse(str(url))
        original_netloc = parsed_url.netloc
        
        # If original_netloc is empty, consider it invalid
        if not original_netloc:
            return None
        
        return original_netloc
    except Exception as e:
        return None  # Return None instead of an empty string

def netloc_status(netloc):
    if netloc is None:
        return 'Absent'
    return 'Present'

In [8]:
# Create 'Netloc' column by applying 'extract_netloc' function
df_new['Netloc'] = df_new['Link'].apply(extract_netloc)

# Create 'Technology_Status' column by applying 'technology_status' function to the 'Netloc' column
df_new['Netloc_Status'] = df_new['Netloc'].apply(netloc_status)

In [9]:
df_new.head()

,File Name,Link,Status,Netloc,Netloc_Status
0,02june-fsa.com,https://static.wixstatic.com/media/,Present,static.wixstatic.com,Present
1,02june-fsa.com,https://static.wixstatic.com/media/0525b5_c4e2...,Present,static.wixstatic.com,Present
2,02june-fsa.com,./#comp-jbgap0ye,Present,None,Absent
3,02june-fsa.com,https://static.parastorage.com/unpkg/focus-wit...,Present,static.parastorage.com,Present
4,02june-fsa.com,https://panorama.wixapps.net/api/v1/bulklog,Present,panorama.wixapps.net,Present


In [10]:
df_new.nunique()

File Name          51644
Link             4421980
Status                 2
Netloc            160479
Netloc_Status          2
dtype: int64

### Exporting Unclean table

df_new.to_csv('Step 1 - Links to Unclean Netloc Table/Unclean_Company_Technography_Relationship.csv', index=False)

***
***

# Creating a New Table having unique combnaion of File Name and Netloc

In [11]:
df_new[df_new['Netloc'].notnull()][['File Name', 'Netloc']].drop_duplicates().nunique()

File Name     50314
Netloc       160479
dtype: int64

In [12]:
# Extract unique combinations of 'File Name' and 'Netloc', excluding null 'Netloc'
df_unique = df_new[df_new['Netloc'].notnull()][['File Name', 'Netloc']].drop_duplicates()

In [13]:
# Add a new column based on substring comparison
df_unique['Netloc_Type'] = df_unique.apply(lambda row: 'Internal' if row['File Name'] in row['Netloc'] else 'External', axis=1)

In [14]:
df_unique.nunique()

File Name       50314
Netloc         160479
Netloc_Type         2
dtype: int64

In [15]:
df_unique.to_csv('Step 1 - Links to Unclean Netloc Table/Unclean_Company_Link_Relationship.csv', index=False)

***
# ***---------------------- END OF STEP 1 ---------------------***
***

***
## ***Step 2 - Cleaning of Netloc Table***
***

In [16]:
#unclean_df = pd.read_csv(r'Step 1 - Links to Unclean Netloc Table\Unclean_Company_Link_Relationship.csv')

In [17]:
df_unique.head()

,File Name,Netloc,Netloc_Type
0,02june-fsa.com,static.wixstatic.com,External
3,02june-fsa.com,static.parastorage.com,External
4,02june-fsa.com,panorama.wixapps.net,External
5,02june-fsa.com,www.02june-fsa.com,Internal
11,02june-fsa.com,www.instagram.com,External


In [18]:
df_unique.count()

File Name      622896
Netloc         622896
Netloc_Type    622896
dtype: int64

## Cleaning Function

In [19]:
def clean_url(url):
    if pd.isna(url) or not isinstance(url, str):
        return None
    
    # Function to clean URL and handle special cases
    
    # Remove special characters from start and end, except for '*.'
    cleaned_url = re.sub(r'^(?![*]\.)[^\w]+|[^\w]+$', '', url)
    
    # Replace '-com.' with '.com.'
    cleaned_url = cleaned_url.replace('-com.', '.com.')
    
    # Remove 'www-', 'www1-', 'www2-', etc. from the beginning
    cleaned_url = re.sub(r'^www\d?-', '', cleaned_url)
    
    # Remove 'www.', 'www1.', 'www2.', etc. from the beginning (in case it wasn't removed by previous step)
    cleaned_url = re.sub(r'^www\d?\.', '', cleaned_url)
    
    # Nullify the value if it does not contain a period, or contains only digits
    if '.' not in cleaned_url or cleaned_url.isdigit():
        return None
    
    # Nullify the value if it contains only numbers and starts with "mail." or "www-"
    if re.match(r'^[\d\W_]+$', cleaned_url) or cleaned_url.startswith('www-'):
        return None
    
    # Convert to numeric and check if it's numeric after cleaning
    try:
        numeric_value = pd.to_numeric(cleaned_url, errors='raise')
        if pd.notna(numeric_value):
            return None
    except (TypeError, ValueError):
        pass
    
    # Remove substring matching the condition if present
    cleaned_url = re.sub(r'.*[\W\d]$', '', cleaned_url)
    
    # Reapply the cleaning to ensure no special characters at start or end except '*.'
    cleaned_url = re.sub(r'^(?![*]\.)[^\w]+|[^\w]+$', '', cleaned_url)
    
    return cleaned_url

In [20]:
clean_df = df_unique.copy()

In [21]:
# Apply the function to the 'url' column and store cleaned values in a new column 'clean_url'
clean_df['Clean_Netloc'] = clean_df['Netloc'].apply(clean_url)

In [22]:
clean_df.nunique()
# 147583
# 147857
# 147823
# 147387

#for actual 147394

File Name        50314
Netloc          160479
Netloc_Type          2
Clean_Netloc    147394
dtype: int64

In [23]:
clean_df_unique = clean_df[clean_df['Clean_Netloc'].notnull()].drop_duplicates(subset=['File Name', 'Clean_Netloc'])

In [24]:
clean_df_unique.nunique()

File Name        50307
Netloc          149711
Netloc_Type          2
Clean_Netloc    147394
dtype: int64

In [25]:
clean_df_unique.to_csv('Step 2 - Unclean Netloc to Clean Netloc/Clean_File_Name_Netloc_relationship_table.csv', index=False)

***
# ***---------------------- END OF STEP 2 ---------------------***
***

***
## ***Step 3 - Filtering the Unique Technography Table***
***

In [26]:
# Define the words to search for
keywords = [
              'connect.facebook.com',
              'business.facebook.com',
              'facebook.com', 'connect.facebook.net', 'm.facebook.com',
              "googletagmanager.com",
              "google-analytics.com",
              "adservice.google.co",
              "googleadservices.com",
              'instagram.com',
              'business.instagram.com',
              'twitter.com',
              'linkedin',
              'amazonaws',
              'microsoft',
              'salesforce',
              # 'dynamic',
              # 'crm',
              'youtube.co',
              'youtube.com',
              '\\*.youtube.com',
              'studio.youtube.com'
       #      'analytics'
       #      'azure',
]

not_keywords = ['cdninstagram', 'platform.instagram.com', 'graph.instagram.com',
       'l.instagram.com',
       'accountscenter.instagram.com', 'api.instagram.com',
       'secure.instagram.com', 'help.instagram.com',
       'images.ak.instagram.com',
       'z-p42.www.instagram.com']
# Create a regex pattern from the keywords
pattern = '|'.join(keywords)
not_pattern = '|'.join(not_keywords)

In [27]:
# Filter the DataFrame based on the pattern
filtered_clean_df_unique = clean_df_unique[clean_df_unique['Clean_Netloc'].str.contains(pattern, case=False, na=False) & ~clean_df_unique['Clean_Netloc'].str.contains(not_pattern, case=False, na=False)]

In [28]:
filtered_clean_df_unique.nunique()

File Name       34786
Netloc           1123
Netloc_Type         2
Clean_Netloc     1086
dtype: int64

In [29]:
filtered_clean_df_unique.to_csv('Step 3 - Clean Netloc to Filtered Clean Netloc/Filtered_Clean_File_Name_Netloc_relationship_table.csv', index=False)

***
# ***---------------------- END OF STEP 3 ---------------------***
***

***
## ***Step 4 - Transforming the Filtered Data***
***

## Transforming Function

In [30]:
def clean_links(df, column_name):
    # Define a function to clean each link based on the conditions
    def clean_link(link):
        if "google.com" in link:
            # Split the link at "google.com" and keep only the part before it and "google.com"
            cleaned_link = link.split("google.com")[0] + "google.com"
            return cleaned_link
        elif "instagram.com" in link:
            # Split the link at "instagram.com" and keep only the part before it and "instagram.com"
            cleaned_link = link.split("instagram.com")[0] + "instagram.com"
            return cleaned_link
        return link

    # Apply the cleaning function to the specified column
    df[column_name] = df[column_name].apply(clean_link)
    
    # Replace links that start with "google.com" or "instagram.com" with the base domain
    df[column_name] = df[column_name].apply(lambda x: "google.com" if x.startswith("google.com") else ("instagram.com" if x.startswith("instagram.com") else x))

    # Replace entire value with "amazonaws.com" if the condition is met
    df.loc[~df['Clean_Netloc'].str.startswith('amazonaws.com') & df['Clean_Netloc'].str.contains('amazonaws.com'), column_name] = "amazonaws.com"
    
    # Dictionary of additional domains to check for and their replacement values
    additional_domains = {
        "googleadservices.com": "googleadservices.com",
        "googletagmanager.com": "googletagmanager.com",
        "googleusercontent.com": "googleusercontent.com",
        "googleapis.com": "googleapis.com",
        ".google.com": "*.google.com",
        "google-anlytics.com": "googleanalytics.com",
        "google-analytics.com": "googleanalytics.com",
        'm.instagram.com': 'instagram.com',
        "connect.facebook": "connect.facebook.com",
        'dynamics.com': 'dynamics.com',
        'youtube.co': 'youtube.com',
        'm.facebook.com': 'facebook.com',
        'adservice.google.co': 'adservice.google.co',
        'microsoft.scloud': 'microsoft.scloud',
        'microsoftcrmportals.com': 'microsoftcrmportals.com',
        'legocrm' : 'legocrm.my.salesforce.com'
    }

    # Replace the whole value with the specified domain if any of the additional conditions are met
    for domain, replacement in additional_domains.items():
        if domain.startswith('.'):
            df.loc[df[column_name].str.contains(domain[1:], regex=True), column_name] = replacement
        else:
            df.loc[df[column_name].str.contains(domain), column_name] = replacement
    
    # Retain specific facebook subdomains and nullify others
    facebook_allowed = ['facebook.com', 'connect.facebook.com', 'm.facebook.com', 'business.facebook.com']
    df.loc[df[column_name].str.contains('facebook') & ~df['Clean_Netloc'].isin(facebook_allowed), column_name] = np.nan
    
    # Additional condition for linkedin.com
    linkedin_allowed = ['*.linkedin.com', 'business.linkedin.com']
    df.loc[df[column_name].str.contains('linkedin') & ~df[column_name].isin(linkedin_allowed), column_name] = 'linkedin.com'
    # Additional condition for Twitter.com

    twitter_allowed = [
        'static.ads-twitter.com', 'twitter.com',
        'mobile.twitter.com', 'business.twitter.com', 'www.twitter.com',
        'm.twitter.com', 'wwww.twitter.com'
    ]
    df.loc[df[column_name].str.contains('twitter') & ~df[column_name].isin(twitter_allowed), column_name] = np.nan

    # Replace specific Twitter subdomains with "twitter.com"
    twitter_specific = ['mobile.twitter.com', 'm.twitter.com', 'wwww.twitter.com']
    df.loc[df[column_name].isin(twitter_specific), column_name] = 'twitter.com'

    return df

In [31]:
transformed_cleaned_df = clean_links(filtered_clean_df_unique, 'Clean_Netloc')

C:\Users\mohit\AppData\Local\Temp\ipykernel_6752\3826111741.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].apply(clean_link)
C:\Users\mohit\AppData\Local\Temp\ipykernel_6752\3826111741.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].apply(lambda x: "google.com" if x.startswith("google.com") else ("instagram.com" if x.startswith("instagram.com") else x))


In [32]:
transformed_cleaned_df

,File Name,Netloc,Netloc_Type,Clean_Netloc
11,02june-fsa.com,www.instagram.com,External,instagram.com
68,02june-fsa.com,www.facebook.com,External,facebook.com
112,0throot.com,www.linkedin.com,External,linkedin.com
156,1-enterprise.com,www.googletagmanager.com,External,googletagmanager.com
208,100kmph.com,www.googletagmanager.com,External,googletagmanager.com
...,...,...,...,...
5418923,zyppys.com,www.twitter.com,External,twitter.com
5418926,zyppys.com,zyppysimages.s3.ap-south-1.amazonaws.com,External,amazonaws.com
5418927,zyppys.com,www.googletagmanager.com,External,googletagmanager.com
5418930,zyppys.com,www.facebook.com,External,facebook.com


In [33]:
transformed_cleaned_df.nunique()

File Name       34786
Netloc           1123
Netloc_Type         2
Clean_Netloc      199
dtype: int64

In [34]:
transformed_cleaned_df.to_csv('Step 4 - Filtered Clean Netloc to Transformed Filtered Clean Netloc/Transformed_Filtered_Clean_File_Name_Netloc_relationship_table.csv', index=False)

***
# ***---------------------- END OF STEP 4 ---------------------***
***

***
## ***Step 5 - Creating the Netloc Table***
***

In [35]:
netloc_table = transformed_cleaned_df[['Clean_Netloc']].dropna().drop_duplicates()

In [36]:
netloc_table

,Clean_Netloc
11,instagram.com
68,facebook.com
112,linkedin.com
156,googletagmanager.com
242,twitter.com
...,...
4567950,technet.microsoft.com
5122171,1ap.my.salesforce.com
5170078,allcargogati.my.salesforce.com
5170158,allcargogati.my.salesforce-sites.com


In [41]:
def assign_technology(df):
    # Define the groups of substrings that map to the same technology
    technology_groups = {
        'Microsoft': {'microsoft.scloud',
                      'microsoftonline',
                      'teams.microsoft',
                      'microsoft365.com'
                    },

        'Google': {'google.com'},
        
        # CLOUD TECHNOLOGY
        'Amazon AWS': {'amazonaws.com'},
        'Microsoft Cloud': {'microsoft.scloud'},
        'Microsoft Azure': {'azure.microsoft'},

        # ADS
        'Google Ads': {'adservice.google.co', 'googleadservices.com', 'googletagmanager', 'googleanalytics'},
        'Facebook Connect': {'connect.facebook'},
        'Microsoft Ads': {'advertise.bingads.microsoft'},
        
        # CRM
        'Salesforce': {'salesforce'},
        'Microsoft CRM': {'microsoftcrm'},
        'Microsoft Dynamics 365': {'dynamics.microsoft'},
        
        # BUSINESS
        'Facebook Business': {'business.facebook.com'},
        'Linkedin Business': {'business.linkedin'},
        'Instagram Business': {'business.instagram'},

        # SOCIAL MEDIA
        'Instagram': {'instagram.com'},
        'Twitter': {'twitter.com'},
        'Facebook': {'facebook.com'},
        'Youtube': {'youtube.com'},
        'Linkedin': {'linkedin.com'},

        # Application Software
        'Microsoft Office': {'office.com'},
        'Microsoft Power BI' :{'powerbi.microsoft'},

        'Microsoft Copilot': {'copilot.microsoft'}

    }
    
    # Create a reverse mapping from substrings to technology
    mapping = {key: technology for technology, keys in technology_groups.items() for key in keys}
    
    # Function to assign technology based on Clean_Netloc
    def get_technology(netloc):
        for key, value in mapping.items():
            if key in netloc.lower():  # Convert to lower case to handle case insensitivity
                return value
        return 'Other'  # Default value if no match is found

    # Create the new column 'technology'
    df['technology'] = df['Clean_Netloc'].apply(get_technology)
    
    return df

In [42]:
netloc_table = assign_technology(netloc_table)

In [52]:
def assign_category(df):
    # Define the groups of technologies that map to the same category
    category_groups = {

        # CRM
        'CRM': {'Salesforce', 'Microsoft CRM', 'Microsoft Dynamics 365'},

        # ADS
        'Ads': {'Microsoft Ads', 'Facebook Connect', 'Google Ads'},

        # SOCIAL MEDIA
        'Social Media': {
            'Youtube',
            'Twitter', 
            'Instagram',
            'Facebook',
            'Linkedin'
        },

        # BUSINESS
        'Business': {
            'Facebook Business',
            'Linkedin Business',
            'Instagram Business'
        },
        

        'Software': {'Microsoft', 'Microsoft Office', 'Google', 'Microsoft Power BI'},

        'Cloud': {'Amazon AWS', 'Microsoft Cloud', 'Microsoft Azure'},
    
        'Other': {'Other'}
    }


    # Create a reverse mapping from technologies to categories
    category_mapping = {tech: category for category, technologies in category_groups.items() for tech in technologies}
    
    # Function to assign category based on technology
    def get_category(technology):
        return category_mapping.get(technology, 'Other')

    # Create the new column 'Category'
    df['Category'] = df['technology'].apply(get_category)
    
    return df

In [53]:
netloc_table = assign_category(netloc_table)

In [54]:
netloc_table.to_csv('Step 5 - Creating Netloc Table/Netloc_table.csv', index=False)